In [ ]:
!pip -q install -U datasets transformers evaluate jiwer huggingface_hub
!apt-get -qq update
!apt-get -qq install -y ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 60.6 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
from datasets import load_dataset, Audio

ds = load_dataset("sukumbasar/ASR_EchoBase_Raw")

ds = ds.cast_column("audio", Audio(decode=False))

print(ds)
print(ds["train"].column_names)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/461 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/10.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'text', 'speaker_id', 'is_common', 'sentence_index'],
        num_rows: 50
    })
})
['audio', 'text', 'speaker_id', 'is_common', 'sentence_index']


In [ ]:
import numpy as np

durations = [
    len(ex["audio"]["bytes"]) for ex in ds["train"]
]

metadata_summary = {
    "num_samples": len(ds["train"]),
}

metadata_summary

{'num_samples': 50}

In [ ]:
TEXT_COL = "text"

In [ ]:
import subprocess, tempfile, numpy as np

def m4a_bytes_to_waveform(audio_bytes, target_sr=16000):
    with tempfile.NamedTemporaryFile(suffix=".m4a") as f:
        f.write(audio_bytes)
        f.flush()

        cmd = [
            "ffmpeg", "-nostdin", "-hide_banner", "-loglevel", "error",
            "-i", f.name,
            "-ac", "1", "-ar", str(target_sr),
            "-f", "s16le", "pipe:1"
        ]
        raw = subprocess.check_output(cmd)

    audio = np.frombuffer(raw, dtype=np.int16).astype(np.float32) / 32768.0
    return audio

In [ ]:
import re, unicodedata

def normalize_text(t):
    if t is None:
        return ""
    t = unicodedata.normalize("NFKC", t).lower()
    t = re.sub(r"[^a-zçğıöşü0-9\s]", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t

In [ ]:
def preprocess_example(example):
    wav = m4a_bytes_to_waveform(example["audio"]["bytes"])
    return {
        "speech": wav,
        "sampling_rate": 16000,
        "text_norm": normalize_text(example[TEXT_COL]),
    }

ds_proc = ds.map(
    preprocess_example,
    remove_columns=ds["train"].column_names
)

print(ds_proc["train"][0].keys())
print(ds_proc["train"][0]["text_norm"])
print(len(ds_proc["train"][0]["speech"]))

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

dict_keys(['speech', 'sampling_rate', 'text_norm'])
bugün hava oldukça sakin
65179


In [ ]:
def is_valid(ex):
    return len(ex["speech"]) > 1600 and len(ex["text_norm"]) > 0  # ~0.1 sn+

ds_proc = ds_proc.filter(is_valid)
print(ds_proc)

Filter:   0%|          | 0/50 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['speech', 'sampling_rate', 'text_norm'],
        num_rows: 50
    })
})


In [ ]:
from huggingface_hub import list_models

models = list_models(search="wav2vec2 turkish ctc", limit=10)
for m in models:
    print(m.modelId)

tgrhn/wav2vec2-turkish-300m
tgrhn/wav2vec2-turkish-300m-2
tgrhn/wav2vec2-turkish-300m-3
tgrhn/wav2vec2-turkish-300m-4
tgrhn/wav2vec2-turkish-300m-5
tgrhn/wav2vec2-turkish-300m-6
tgrhn/wav2vec2-turkish-300m-7
tgrhn/wav2vec2-turkish-300m-8
tgrhn/wav2vec2-turkish-1
tgrhn/wav2vec2-turkish-2


In [ ]:
import torch
from transformers import AutoProcessor, AutoModelForCTC

MODEL_ID = "m3hrdadfi/wav2vec2-large-xlsr-turkish"

processor = AutoProcessor.from_pretrained(MODEL_ID)
model = AutoModelForCTC.from_pretrained(MODEL_ID)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

print("device:", device)

preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/configuration_utils.py:335: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


vocab.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

device: cuda


In [ ]:
ex = ds_proc["train"][0]

inputs = processor(
    ex["speech"],
    sampling_rate=16000,
    return_tensors="pt",
    padding=True
)
inputs = {k: v.to(device) for k, v in inputs.items()}

with torch.no_grad():
    logits = model(**inputs).logits

pred_ids = torch.argmax(logits, dim=-1)
pred_text = processor.batch_decode(pred_ids)[0]

print("REF:", ex["text_norm"])
print("HYP:", normalize_text(pred_text))

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

REF: bugün hava oldukça sakin
HYP: bugün hava oldukça sakin


In [ ]:
from evaluate import load
wer_metric = load("wer")

def compute_wer(n=50, split="train"):
    refs, hyps = [], []
    for i in range(min(n, len(ds_proc[split]))):
        ex = ds_proc[split][i]
        inputs = processor(
            ex["speech"],
            sampling_rate=16000,
            return_tensors="pt",
            padding=True
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            logits = model(**inputs).logits

        pred_ids = torch.argmax(logits, dim=-1)
        pred_text = processor.batch_decode(pred_ids)[0]

        refs.append(ex["text_norm"])
        hyps.append(normalize_text(pred_text))

    return wer_metric.compute(predictions=hyps, references=refs)

print("WER@50:", compute_wer(n=50))

WER@50: 0.35978835978835977
